# Simulate travel time between two points for different transport modes

In [1]:
import haversine as hs
import folium
import math
import sys
from pathlib import Path
from station_utilities import *
from simulations import *

In [2]:
def init_stationfinder() -> StationFinder:
    """
    Initializes the StationFinder utility by loading Citibike and subway graph data.

    Returns:
        StationFinder: An initialized StationFinder object.
    
    Raises:
        SystemExit: If required graph files (.gml) are not found.
    """
    citibike_weekday = '../citibike_weekday_network.gml'
    citibike_weekend = '../citibike_weekend_network.gml'
    subway_graph = '../subway_graph_weekday_weekend.gml'

    citibike_path = citibike_weekday if Path(citibike_weekday).exists() else citibike_weekend

    if not Path(citibike_path).exists():
            print("Error: Citibike graph not found. Please run citibike_processor.py first.")
            sys.exit(1)
        
    if not Path(subway_graph).exists():
        print(f"Error: Subway graph not found at {subway_graph}")
        sys.exit(1)

    print("Initializing StationFinder...")
    try:
        finder = StationFinder(
            citibike_graph_path=citibike_path,
            subway_graph_path=subway_graph
        )
    except Exception as e:
        print(f"Error initializing StationFinder: {e}")
        print("\nIf subway coordinates are not available, you can:")
        print("1. Provide a CSV file with subway coordinates")
        print("2. Download MTA GTFS data manually from: http://web.mta.info/developers/data/nyct/subway/google_transit.zip")
        sys.exit(1)

    return finder

In [3]:
station_finder = init_stationfinder()

Initializing StationFinder...
Successfully loaded 381 subway stations
Successfully loaded 381 subway stations


In [4]:
def simulate(src_hub_center, dst_hub_center, station_finder, radius_km=1.0, num_points=100, use_batch_api=True):
    m = None

    simulator = Simulator(finder=station_finder)

    print(f'Generating {num_points} random starting points:')
    src_hub_points = simulator.generate_random_points(src_hub_center, radius_km=radius_km, num_points=num_points)
    print(f'Generating {num_points} random destination points:')
    dst_hub_points = simulator.generate_random_points(dst_hub_center, radius_km=radius_km, num_points=num_points)

    print("Inserting virtual station at source hub center...")
    simulator.insert_station(('virtual-station', src_hub_center))

    # --- Calculate for Bike Transit ---
    print("" + "="*80)
    print('\nSimulating Citibike journeys...\n')
    print("" + "="*80)
    bike_travel_time_matrix, bike_route_matrix = simulator.calculate_travel_time_matrix(src_hub_points, dst_hub_points, radius_km=radius_km, mode='citibike', use_batch_api=use_batch_api)

    # --- Calculate for Subway Transit ---
    print("" + "="*80)
    print('\nSimulating subway journeys...\n')
    print("" + "="*80)
    subway_travel_time_matrix, subway_route_matrix = simulator.calculate_travel_time_matrix(src_hub_points, dst_hub_points, radius_km=radius_km, mode='subway', use_batch_api=use_batch_api)

    results = {
        'bike_travel_time_matrix': bike_travel_time_matrix,
        'bike_route_matrix': bike_route_matrix,
        'subway_travel_time_matrix': subway_travel_time_matrix,
        'subway_route_matrix': subway_route_matrix
    }

    # create a map visualization of the results
    # m = simulator.map_simulation_results(bike_route_matrix, subway_route_matrix, src_hub_points, dst_hub_points)

    return results, m

In [5]:
def calculate_average_travel_times(results):
    bike_tt_matrix, subway_tt_matrix = results['bike_travel_time_matrix'], results['subway_travel_time_matrix']

    # Calculate the mean of all values in the bike travel time matrix
    avg_bike_time_seconds = np.mean(bike_tt_matrix)

    # Calculate the mean of all values in the subway travel time matrix
    avg_subway_time_seconds = np.mean(subway_tt_matrix)

    return avg_bike_time_seconds, avg_subway_time_seconds

In [ ]:
def run_simulation(hubs, radius_km=1.0, num_points=100, use_batch_api=True):
    dst_hub_center = (40.73596286967174, -73.99050483291762)

    bike_results = dict()
    subway_results = dict()

    for i, hub in enumerate(hubs):
        print("" + "="*80)
        print(f"\nRunning simulations for hub {i+1}/{len(hubs)} at location: {hub}\n")
        print("" + "="*80)
        results, m = simulate(hub, dst_hub_center, station_finder, radius_km=radius_km, num_points=num_points, use_batch_api=use_batch_api)
        avg_bike_time, avg_subway_time = calculate_average_travel_times(results)
        avg_bike_time_seconds = f"{avg_bike_time % 60:.0f}" if avg_bike_time % 60 >= 10 else f"0{avg_bike_time % 60:.0f}"
        avg_subway_time_seconds = f"{avg_subway_time % 60:.0f}" if avg_subway_time % 60 >= 10 else f"0{avg_subway_time % 60:.0f}"
        bike_results[hub] = f"{math.floor(avg_bike_time / 60)}:{avg_bike_time_seconds}"
        subway_results[hub] = f"{math.floor(avg_subway_time / 60)}:{avg_subway_time_seconds}"

    bike_avg_times = pd.DataFrame.from_dict({'Average Travel Time By Bike': bike_results})
    subway_avg_times = pd.DataFrame.from_dict({'Average Travel Time By Metro': subway_results})
    print("" + "="*80)
    print(f"\nSimulation for {hub} complete!\n")
    print("" + "="*80)

    return bike_avg_times, subway_avg_times

In [ ]:
hubs = [(40.6535720712609, -73.931131331664), (40.67736425375234, -74.01022251723293), (40.759128917773964, -73.88691842716244)] # utica ave brooklyn, redhook, Jackson heights

radius_km = 1.0
num_points = 20
use_batch_api = True # use google maps distance matrix api instead of directions api

bikes_results_df, subway_results_df = run_simulation(hubs, radius_km=radius_km, num_points=num_points, use_batch_api=True)

# print("--- Average Travel Times for All Pairs ---")
# print(f"Average Bike Travel Time: {math.floor(avg_bike_time_seconds / 60)} minutes and {avg_bike_time_seconds % 60:.0f} seconds")
# print(f"Average Subway Travel Time: {math.floor(avg_subway_time_seconds / 60)} minutes and {avg_subway_time_seconds % 60:.0f} seconds")


Running simulations for hub 0/3 at location: (40.6535720712609, -73.931131331664)

Generating 20 random starting points:
Generating 20 random destination points:
Inserting virtual station at source hub center...
Inserting new station: virtual-station at (40.6535720712609, -73.931131331664)

Simulating Citibike journeys...

Pre-calculating nearest citibike stations...
Using batched Distance Matrix API calls...
  - Calculating walk times to origin stations...
Using batched Distance Matrix API calls...
  - Calculating walk times to origin stations...
  - Calculating bike times between stations...
  - Calculating bike times between stations...
  - Calculating walk times from destination stations...
  - Calculating walk times from destination stations...
 - Combining travel time legs...

Simulating subway journeys...

Pre-calculating nearest subway stations...
 - Combining travel time legs...

Simulating subway journeys...

Pre-calculating nearest subway stations...
Using batched Distance 

In [ ]:
bikes_results_df.to_csv('simulation_results/citibike.csv')
bikes_results_df.head()

,,Average Travel Time By Bike
40.653572,-73.931131,60:08
40.677364,-74.010223,45:31
40.759129,-73.886918,45:39


In [ ]:
subway_results_df.to_csv('simulation_results/subway.csv')
subway_results_df.head()

,,Average Travel Time By Metro
40.653572,-73.931131,40:13
40.677364,-74.010223,30:07
40.759129,-73.886918,34:42
